# 📈 Quant Apprentice: An Agentic AI for Investment Research

This notebook is the final implementation of the Quant Apprentice, an autonomous agent designed to perform comprehensive financial
analysis on a publicly traded company. 
The agent follows a sophisticated, multi-phase workflow that mirrors a real-world research team.

**Core Agentic Capabilities Demonstrated:**
- **Planning**: The agent executes a pre-defined, multi-step plan from data gathering to final report generation.
- **Dynamic Tool Use**: It leverages external APIs for financial, news, and macroeconomic data.
- **Self-Reflection**: It uses an Evaluator-Optimizer loop to critique and refine its own analysis.
- **Learning**: It saves its final analysis to a memory directory for future reference.

This notebook will now execute the agent's full workflow for a selected company.

In [1]:
import os
import json
from datetime import datetime
from dotenv import load_dotenv
import google.generativeai as genai
from IPython.display import display, Markdown

# --- Import all project modules ---
# This demonstrates the modular design of the project.
from src.tools.financial_data_fetcher import get_stock_fundamentals, get_macro_economic_data
from src.tools.news_fetcher import get_company_news
from src.workflows.news_analysis_chain import analyze_article_chain
from src.workflows.specialist_router import route_and_execute_task
from src.workflows.report_evaluator import generate_and_evaluate_report
from src.memory.vector_memory import VectorMemory

# --- Load Environment Variables and Configure APIs ---
load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

print("Environment is set up. All modules are imported.")

Environment is set up. All modules are imported.


In [2]:
# --- Configure the Agent's Target and LLM ---

# You can change these values to research a different company
COMPANY_TICKER = "NVDA"
COMPANY_NAME = "NVIDIA"

# Initialize the Gemini Model
# Using a lower temperature for more deterministic, factual output
llm_config = {'temperature': 0.2}
gemini_pro_model = genai.GenerativeModel(
    'gemini-2.5-pro',
    generation_config=llm_config
)

print(f"🤖 Quant Apprentice agent configured to analyze: {COMPANY_NAME} ({COMPANY_TICKER})")

🤖 Quant Apprentice agent configured to analyze: NVIDIA (NVDA)


In [3]:
print(f"\n--- [Agent Start]: Beginning analysis for {COMPANY_NAME}... ---")

# --- PHASE 1: AUTONOMOUS DATA GATHERING ---
# DEMONSTRATION: The agent uses its tools to fetch data from multiple external sources.
print("\n[Phase 1]: Gathering data from all sources...")

fred_api_key = os.getenv("FRED_API_KEY")
news_api_key = os.getenv("NEWS_API_KEY")

financial_data = get_stock_fundamentals(COMPANY_TICKER)
macro_data = get_macro_economic_data(fred_api_key)
news_data = get_company_news(COMPANY_NAME, news_api_key, num_articles=3) # Fetch 3 articles for broader context

print("✅ [Phase 1]: Data gathering complete.")


# --- PHASE 2: DATA PROCESSING & SPECIALIST ANALYSIS ---
print("\n[Phase 2]: Processing and routing data to specialist analysts...")

# DEMONSTRATION: Workflow 1 - Prompt Chaining
# The agent processes each news article to create a structured analysis.
processed_news_analyses = []
if "error" in news_data or not news_data.get("articles"):
    print("⚠️ Could not retrieve news. Skipping news analysis.")
    news_impact_analysis = "No news available for analysis."
else:
    for article in news_data["articles"]:
        analysis = analyze_article_chain(article['content'], gemini_pro_model)
        if "error" not in analysis:
            processed_news_analyses.append(analysis)
    
    # DEMONSTRATION: Workflow 2 - Routing
    # The agent sends the processed news to the News Analyst specialist.
    news_impact_analysis = route_and_execute_task('analyze_news_impact', {"news_items": processed_news_analyses}, gemini_pro_model)

# The agent routes the other data to their respective specialists.
financial_analysis = route_and_execute_task('analyze_financials', financial_data, gemini_pro_model)
market_context_analysis = route_and_execute_task('analyze_market_context', macro_data, gemini_pro_model)

print("✅ [Phase 2]: All specialist analyses are complete.")


# --- PHASE 3: SYNTHESIS, EVALUATION, AND REFINEMENT ---
print("\n[Phase 3]: Synthesizing and refining the final report...")

# DEMONSTRATION: Workflow 3 - Evaluator-Optimizer Loop & Agent Self-Reflection
# The agent now synthesizes the specialist reports, critiques its own work, and refines the final output.
final_outputs = generate_and_evaluate_report(
    company_name=COMPANY_NAME,
    financial_analysis=financial_analysis,
    news_impact_analysis=news_impact_analysis,
    market_context_analysis=market_context_analysis,
    llm=gemini_pro_model
)

print("✅ [Phase 3]: Self-reflection and refinement complete.")
print("\n--- [Agent Finish]: Analysis complete. Displaying final report. ---")


--- [Agent Start]: Beginning analysis for NVIDIA... ---

[Phase 1]: Gathering data from all sources...
--- [Tool Action]: Fetching fundamental data for NVDA... ---
--- [Tool Success]: Successfully fetched fundamentals for NVDA. ---
--- [Tool Action]: Fetching macroeconomic data from FRED... ---
--- [Tool Success]: Successfully fetched macroeconomic data. ---
[Tool Action]: Fetching top 3 news articles for NVIDIA...
[Tool Success]: Successfully fetched 3 articles.
✅ [Phase 1]: Data gathering complete.

[Phase 2]: Processing and routing data to specialist analysts...
--- [Workflow Action]: Starting Refined News Analysis Chain... ---
--- [Workflow Success]: Refined News Analysis completed. ---
--- [Workflow Action]: Starting Refined News Analysis Chain... ---
--- [Workflow Success]: Refined News Analysis completed. ---
--- [Workflow Action]: Starting Refined News Analysis Chain... ---
--- [Workflow Success]: Refined News Analysis completed. ---
--- [Router]: Routing to News Analyst... --

In [4]:
# --- Display the Final, Multi-Stage Report ---
# This cell clearly shows the agent's iterative improvement process.

if "error" in final_outputs:
    display(Markdown(f"**An error occurred during the final stage:** {final_outputs['error']}"))
else:
    display(Markdown(f"# Final Investment Report: {COMPANY_NAME} ({COMPANY_TICKER})"))
    
    display(Markdown("---"))
    display(Markdown("## Initial Draft Report (by Chief Investment Strategist)"))
    display(Markdown(final_outputs['draft_report']))
    
    display(Markdown("---"))
    display(Markdown("## Skeptical Feedback (by Risk Manager)"))
    display(Markdown(final_outputs['feedback']))
    
    display(Markdown("---"))
    display(Markdown("## **Final, Refined Report**"))
    display(Markdown(final_outputs['final_report']))

# Final Investment Report: NVIDIA (NVDA)

---

## Initial Draft Report (by Chief Investment Strategist)

Of course. Here is the synthesized investment report for NVIDIA.

***

### **Investment Report: NVIDIA Corporation (NVDA)**

**To:** Investment Committee
**From:** Chief Investment Strategist
**Date:** October 26, 2023
**Subject:** Investment Recommendation for NVIDIA (NVDA)

---

#### **1. Executive Summary**

NVIDIA presents a classic case of a high-quality, financially robust company whose premium valuation is facing significant macroeconomic headwinds. While the company's fundamentals and strategic positioning are superb, the current high-interest-rate environment creates considerable valuation risk, leading us to a cautious stance.

#### **2. Key Findings**

*   **Premium Valuation:** The stock trades at a high Trailing P/E of 52.1, indicating that significant future growth is already priced in by the market. (**Quantitative Financial Analysis**)
*   **Exceptional Financial Health:** NVIDIA maintains a strong balance sheet with a net cash position (Enterprise Value < Market Cap), affording it significant financial flexibility for reinvestment and R&D. (**Quantitative Financial Analysis**)
*   **Unfavorable Macro Environment:** The current high-interest-rate environment (Effective Fed Funds Rate at 4.22%) creates direct headwinds for high-growth technology stocks by increasing the discount rate applied to future earnings. (**Macroeconomic Context**)
*   **Strategic Supply Chain Management:** NVIDIA's decision to engage Intel as a foundry customer is a positive strategic development, validating its operational foresight and diversifying its manufacturing base. (**News Impact Analysis**)

#### **3. Final Recommendation**

Recommendation: **Hold**

#### **4. Justification**

*   **Exceptional Company Quality vs. Stretched Valuation:** NVIDIA’s fundamentals are undeniable. The **Quantitative Analysis** shows a financially healthy company with strong anticipated earnings growth. However, its P/E ratio of 52.1 is at a significant premium, suggesting the stock is fully valued and vulnerable to any potential earnings miss.

*   **Macroeconomic Pressure on Growth Stocks:** The **Macroeconomic Context** report explicitly identifies high-growth technology as a sector facing headwinds from restrictive monetary policy. High interest rates put downward pressure on the valuations of companies like NVIDIA, whose value is heavily derived from long-term future earnings.

*   **Positive Catalysts are Insufficient to Overcome Near-Term Risks:** While positive news, such as the strategic partnership with Intel (**News Impact Analysis**), reinforces the long-term bull case, it is not enough to justify adding new capital at this valuation in the current market. A 'Hold' rating acknowledges the company's superior quality while respecting the significant valuation and macroeconomic risks that limit near-term upside potential.

---

## Skeptical Feedback (by Risk Manager)

Of course. As a skeptical Risk Manager, here is my constructive feedback on the draft report.

*   **The valuation analysis relies heavily on a backward-looking metric.** Citing a Trailing P/E of 52 for a company experiencing exponential earnings growth can be misleading. The core of the investment debate is the *sustainability* of that growth. A stronger analysis would incorporate forward P/E estimates or a PEG ratio to assess if the valuation is truly stretched relative to its future, not its past, earnings power.

*   **The report overlooks significant company-specific risks in favor of a generic macroeconomic headwind.** The primary risks to NVIDIA are arguably geopolitical (e.g., US-China chip restrictions impacting a key market) and competitive (e.g., AMD's progress, major customers developing in-house AI chips). These threats are more direct and material than the broad impact of interest rates on discount models.

*   **The analysis lacks balance by not quantifying the asymmetric risk profile.** The "Hold" recommendation focuses on preventing downside from a valuation correction but doesn't adequately weigh the opportunity cost of missing a continued surge driven by the AI super-cycle. A robust risk assessment would model both a "macro-driven correction" scenario and a "structural growth" scenario to better frame the potential outcomes.

---

## **Final, Refined Report**

Of course. Here is the revised investment report for NVIDIA, incorporating the Risk Manager's feedback to create a more robust and balanced analysis.

---

### **Final Investment Report: NVIDIA Corporation (NVDA)**

**To:** Investment Committee
**From:** Chief Investment Strategist
**Date:** October 26, 2023
**Subject:** Revised Investment Thesis and Recommendation for NVIDIA (NVDA)

---

### **Executive Summary**

This report provides a revised investment recommendation for NVIDIA Corporation (NVDA). Our analysis concludes that NVIDIA is at the epicenter of a structural shift in technology driven by Artificial Intelligence, affording it a dominant market position and unprecedented growth prospects. This is reflected in its robust financial health and a strategic focus on reinvesting capital to solidify its leadership.

However, the company's premium valuation is predicated on the flawless execution and long-term sustainability of this growth. This optimism faces significant company-specific risks, including geopolitical tensions surrounding semiconductor supply chains and an intensifying competitive landscape. Furthermore, the restrictive macroeconomic environment poses a latent threat to all high-growth equities.

Given the profound asymmetry in potential outcomes—the opportunity cost of missing a generational tech boom versus the risk of a sharp valuation correction—we are upgrading our rationale while maintaining a **Hold** recommendation. This position allows current investors to retain exposure to the powerful AI narrative while acknowledging the material risks that could derail the current trajectory.

### **Key Findings**

1.  **Dominant Market Position in AI Super-Cycle:** NVIDIA's GPUs are the foundational technology for the current AI revolution. The company has a significant technological moat, a deep software ecosystem (CUDA), and is capturing the vast majority of spending in the AI training and inference markets, leading to exponential earnings growth.

2.  **Forward-Looking Valuation Demands Sustained Growth:** While a Trailing P/E of 52.1 appears high, the more relevant Forward P/E of 44.5 indicates significant near-term earnings growth is already priced in. The investment debate hinges on the sustainability of this growth trajectory. The valuation is less a reflection of past performance and more a market expectation of continued, multi-year dominance.

3.  **Robust Financial Health and Reinvestment Focus:** The company's balance sheet is exceptionally strong, evidenced by a net cash position (Enterprise Value < Market Cap). A negligible dividend yield (0.02%) and low payout ratio (1.14%) underscore a disciplined strategy of reinvesting nearly all earnings into R&D and expansion to capitalize on the AI opportunity.

4.  **Significant Company-Specific Risks:** The primary threats to the investment thesis are not generic market headwinds but are specific to NVIDIA's operations:
    *   **Geopolitical Risk:** US-China trade restrictions on advanced semiconductors directly impact a key geographic market and create supply chain uncertainty.
    *   **Competitive Risk:** Competitors like AMD are intensifying their efforts in the AI space, while major customers (e.g., Google, Amazon, Microsoft) are developing in-house AI chips, which could reduce their long-term reliance on NVIDIA.

### **Recommendation: Hold**

We recommend a **Hold** rating for NVIDIA Corporation (NVDA).

### **Justification**

The "Hold" recommendation is based on a careful balancing of two powerful, divergent scenarios. The current market environment for NVIDIA is characterized by extreme potential outcomes, making a neutral stance the most prudent strategic position.

**1. The Bull Case: The "Structural Growth" Scenario**
The argument for continued upside is compelling. If the AI super-cycle is still in its early innings, NVIDIA's growth could continue to exceed already high expectations. In this scenario, the company's dominant market share, technological leadership, and expanding software ecosystem would allow it to compound earnings at a rate that makes the current valuation appear reasonable in hindsight. The opportunity cost of not being invested in what could be a generational technology leader is substantial.

**2. The Bear Case: The "Valuation & Macro Correction" Scenario**
Conversely, the risks are tangible and significant. This scenario could be triggered by several factors:
*   **Geopolitical Escalation:** Further restrictions on chip sales to China could materially impact revenue forecasts.
*   **Competitive Erosion:** A viable competitor gaining market share or a major customer successfully shifting to an in-house solution would challenge the narrative of NVIDIA's unassailable moat.
*   **Macroeconomic Pressure:** While company-specific factors are currently dominant, the high-interest-rate environment makes the entire market sensitive to negative catalysts. Any sign of slowing growth at NVIDIA could trigger a rapid and severe valuation de-rating, as its premium valuation offers little margin for error.

**Conclusion:**
The "Hold" recommendation acknowledges the immense potential outlined in the structural growth scenario while respecting the clear and present dangers identified in the correction scenario. For existing shareholders, it advises against taking profits prematurely in a powerful secular trend. For new investors, it cautions against initiating a position at a peak valuation when significant geopolitical and competitive risks remain unresolved. We will continue to monitor competitive developments and geopolitical news closely for a catalyst that could shift the risk/reward balance decisively in one direction.

In [5]:
# --- PHASE 4: AGENT LEARNING ---
# DEMONSTRATION: The agent saves its final, refined output to a file, simulating memory.
# This allows the agent to "learn" from its analysis for future reference.

print("\n[Phase 4]: Saving final analysis to memory...")

# Create the memory directory if it doesn't exist
if not os.path.exists("memory"):
    os.makedirs("memory")

# Define a unique filename
# --- PHASE 4: AGENT LEARNING (VECTOR MEMORY) ---
# DEMONSTRATION: The agent saves its final, refined output to a vector database.
# This allows the agent to perform semantic searches on its past analyses.

print("\n[Phase 4]: Saving final analysis to vector memory...")

# Initialize the memory system
memory = VectorMemory()

# Add the latest analysis
memory.add_analysis(COMPANY_TICKER, final_outputs.get('final_report', 'No report generated.'))

# Perform a test query to show the memory is working
test_query = f"What was my most recent analysis of {COMPANY_NAME}?"
query_results = memory.query_memory(test_query, n_results=1)

if query_results:
    print("\n--- Memory Query Test ---")
    display(Markdown(f"**Query:** '{test_query}'"))
    display(Markdown("**Result:**"))
    display(Markdown(query_results[0]))


[Phase 4]: Saving final analysis to memory...

[Phase 4]: Saving final analysis to vector memory...
[Memory]: Initializing ChromaDB at memory/chroma_db
[Memory]: Adding analysis for NVDA to vector memory...
[Memory]: Successfully added document with ID: NVDA_2025-10-18-11:08:45
[Memory]: Querying memory with: 'What was my most recent analysis of NVIDIA?'

--- Memory Query Test ---


**Query:** 'What was my most recent analysis of NVIDIA?'

**Result:**


NVIDIA's main strength is its dominance in the AI chip market. However, a key risk is its high valuation 
and increasing competition from other tech giants in the semiconductor space.
